In [1]:
#Got a working training session from start to finish
#Good result but this used pre-trained model

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
from src import data_download
data_download.fetch_data()

Folder already has folders 'annotations' and 'images'.
Assuming you already have the data and skipping fetch.


In [3]:
import torch
from torchvision import transforms, datasets
import torch.nn as nn
from torchvision.io import read_image, ImageReadMode
from torch.utils.data import Dataset
import torch.optim as optim

In [4]:
DATA_DIR = 'data/images'

In [5]:
from src import data_handling
train, test, val = data_handling.get_target_dfs(train=0.6, test=0.2, val=0.2)

In [29]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_df, img_dir, transform=None):
        self.img_labels = annotations_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_idx = self.img_labels.index[idx]
        img_path = os.path.join(self.img_dir, "im"+str(img_idx)+".jpg")
        image = read_image(img_path, ImageReadMode.RGB).float().to("cuda")
        labels = torch.from_numpy(self.img_labels.iloc[idx].values).float().to("cuda")
        if self.transform:
            image = self.transform(image)
        return image, labels

In [31]:
train_loader = torch.utils.data.DataLoader(dataset=CustomImageDataset(train, DATA_DIR, transform=None), batch_size=50, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=CustomImageDataset(test, DATA_DIR, transform=None), batch_size=50, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=CustomImageDataset(val, DATA_DIR, transform=None), batch_size=50, shuffle=True)

In [32]:
from torchvision.models import resnet50, ResNet50_Weights

In [33]:
class MultiLabelClassifier(nn.Module):
    def __init__(self, num_labels=14):
        super(MultiLabelClassifier, self).__init__()
        self.base_model = resnet50(weights=ResNet50_Weights.DEFAULT)
        self.classifier = nn.Sequential(
            nn.Linear(1000, 256),
            nn.ReLU(),
            nn.Linear(256, num_labels),
            nn.Sigmoid()
        )

    def forward(self, x):
        features = self.base_model(x)
        out = self.classifier(features)
        return out

In [34]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [35]:
model = MultiLabelClassifier().to(device)

In [36]:
loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [37]:
epochs=5
for epoch in range(epochs):
    train_loss = 0
    train_correct = 0
    total = 0
    for batch_num, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
    print(f"Epoch {epoch} done. Training loss {train_loss/(batch_num+1):.3f}")

Epoch 0 done. Training loss 0.928
Epoch 1 done. Training loss 0.908
Epoch 2 done. Training loss 0.884
Epoch 3 done. Training loss 0.851
Epoch 4 done. Training loss 0.816


In [38]:
with torch.no_grad():
    accuracy = None
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted_labels = (outputs > 0.5).int()
        _accuracy = (predicted_labels == labels).float().mean().item()
        if not accuracy:
            accuracy = _accuracy
        else:
            accuracy = (accuracy + _accuracy) / 2

print(f"Test accuracy: {accuracy:.3f}")

Test accuracy: 0.931
